In [2]:
#Shaun Richardson
#CS-499-T5477
#May 19, 2021

from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#Import crud module
from crud import BidSystem




###########################
# Data Manipulation / Model
###########################
#Hard coded Login
username = "biduser"
password = "password"
items = BidSystem(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(items.readAll({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#Auction logo
image_filename = 'logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#App HTML layout
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.A([
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style={
                    'height' : '25%',
                    'width' : '25%',
                    'float' : 'left',
                    'position' : 'relative',
                    'padding-top' : 0,
                    'padding-right' : 0
                })
    ], href='https://www.snhu.edu'),
    html.Br(),
    html.Hr(),
    html.Center(html.B(html.H1('Shaun Richardson SNHU CS-499 Capstone Dashboard'))),
    html.Hr(),
#Dropdown selection
    dcc.Dropdown(
        id='demo-dropdown',
        options=[
            {'label': 'All', 'value': 'All'},
            #{'label': '', 'value': ''},
        ],
        value='All'
    ),
#datatable Options
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #features for your interactive data table to make it user-friendly
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=True,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current = 0,
        page_size= 10,
        ),
    html.Hr(),
    html.Div(id='output'),
    html.Br(),
    html.Hr(),
#This sets up the dashboard for the pie chart",
    html.Div(className='row',
        style={'display' : 'flex'},
        children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
         
            ),
        ]),
    html.Br(),
    html.Hr(),
    html.Header("Shaun Richardson SNHU CS-499 Capstone Dashboard")
    ])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('demo-dropdown', 'value')])
def update_dashboard(value):
###filter interactive data table with MongoDB queries
    if (value == 'All'):
        df = pd.DataFrame.from_records(items.readAll({}))
   
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
        
    return (data,columns)

#callback that controls the color of selected item
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#Callback to show which row was deleted (trying to use to trigger a delete)
@app.callback(Output('output', 'children'),
              [Input('datatable-id', 'data_previous')],
              [State('datatable-id', 'data')])
def show_removed_rows(previous, current):
    if previous is None:
        dash.exceptions.PreventUpdate()
    else:
        return [f'Just removed {row}' for row in previous if row not in current]
    #[f'Just removed {row}' for row in previous if row not in current] 
    #items.delete(items, row)




#Callback for the pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)

    return [
       dcc.Graph(
           id='graph_id',
           figure = px.pie(dff, 
                           values='Auction ID', 
                           names='Department',
                           title=''
                          ),
           )]   
                                


app
